In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
df=pickle.load(open('dataset_level_2.pkl','rb'))

Now we need 
1. batting_team
2. bowling_team
3. city
4. current score
5. balls left
6. wickets left
7. current run rate
8. last 10 overs runs

In [3]:
# lets focus on City
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed        0
city                56082
venue                   0
dtype: int64

we can see that 56,082 city values are null
that's why we take venue
we also observe that there are no null values in venue column
that means we have the venue for missing city 

In [4]:
df[df['city'].isnull()]['venue'].value_counts()

Sydney Cricket Ground                      9595
Rangiri Dambulla International Stadium     9450
Melbourne Cricket Ground                   7778
Adelaide Oval                              7166
Dubai International Cricket Stadium        6314
Pallekele International Cricket Stadium    5945
Sharjah Cricket Stadium                    4291
Harare Sports Club                         1538
Chittagong Divisional Stadium               900
Multan Cricket Stadium                      883
Rawalpindi Cricket Stadium                  617
Perth Stadium                               526
Dubai Sports City Cricket Stadium           518
Sharjah Cricket Association Stadium         303
Queenstown Events Centre                    258
Name: venue, dtype: int64

these are the counts of those venues for whose city is null
now,
the plan is to extract only the first word of these venue & use it those missing city column

In [5]:
cities=np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

if city is null then extract the first word of venue & fill it in the city coloumn
otherwise 
city remains as it is

this is the meaning of the code

In [6]:
df['city']=cities

In [7]:
# checking for further null values
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

There is no null values now

In [8]:
# now there is no need of venue column 
# so we drop it from the dataframe
df.drop(columns=['venue'],inplace=True)

In [9]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Pakistan,0.1,0,0,Brisbane
1,2,Australia,Pakistan,0.2,0,0,Brisbane
2,2,Australia,Pakistan,0.3,0,0,Brisbane
3,2,Australia,Pakistan,0.4,0,0,Brisbane
4,2,Australia,Pakistan,0.5,1,0,Brisbane
...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele


# Now City is Categorical Variable


In [10]:
df['city'].value_counts()

Mirpur        18663
London        16581
Colombo       15967
Sydney        10205
Abu Dhabi     10003
              ...  
Peshawar        307
Faridabad       305
Bulawayo        302
Jamshedpur      300
Dharmasala      271
Name: city, Length: 97, dtype: int64

# There are 97 cities where matches had been played
# There should be so city where only one match had been played
# that's why we can't consider all the cities
# We consider only those cities where minimum 5 matches were played

In [11]:
df['city'].value_counts()[df['city'].value_counts()>=1500].shape

(53,)

there are 53 cities where minimum 5 matches were played

In [12]:
df['city'].value_counts()[df['city'].value_counts()>=1500].sum()

276744

now we have only 2,76,744 rows left

In [13]:
eligible_cities=df['city'].value_counts()[df['city'].value_counts()>=1500].index.tolist()

In [14]:
df=df[df['city'].isin(eligible_cities)]

In [15]:
# now we focus on current score
df['current_score']=df.groupby('match_id').cumsum()['runs']

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\3148008916.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['current_score']=df.groupby('match_id').cumsum()['runs']
C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\3148008916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_score']=df.groupby('match_id').cumsum()['runs']


In [16]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1
...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195


In [17]:
# now we focus on balls_left & wickets_left
# firstly we split the ball coloumn with repect to decimal
df['over']=df['ball'].apply(lambda x:str(x).split('.')[0])
df['ball_no']=df['ball'].apply(lambda x:str(x).split('.')[1])



C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\655071928.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['over']=df['ball'].apply(lambda x:str(x).split('.')[0])
C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\655071928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ball_no']=df['ball'].apply(lambda x:str(x).split('.')[1])


In [18]:
df['balls_bowled']=(df['over'].astype('int')*6)+df['ball_no'].astype('int')

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\2979137193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_bowled']=(df['over'].astype('int')*6)+df['ball_no'].astype('int')


In [19]:
df['balls_left']=300-df['balls_bowled']
df['balls_left']=df['balls_left'].apply(lambda x:0 if x<0 else x)

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\19006928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left']=300-df['balls_bowled']
C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\19006928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left']=df['balls_left'].apply(lambda x:0 if x<0 else x)


In [20]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295
...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,0,Pallekele,194,39,5,239,61
245,1795,Sri Lanka,Australia,39.6,0,0,Pallekele,194,39,6,240,60
246,1795,Sri Lanka,Australia,39.7,1,0,Pallekele,195,39,7,241,59
247,1795,Sri Lanka,Australia,40.1,0,0,Pallekele,195,40,1,241,59


In [21]:
# we can see that if for any ball wicket happens then the dismissing batsman name wii be there
# so we do one thing
# for those records we substitute batsman name to 1
df['player_dismissed']=df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed']=df['player_dismissed'].astype('int')
df['player_dismissed']=df.groupby('match_id').cumsum()['player_dismissed']
df['wickets_left']=10-df['player_dismissed']

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\4268819930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed']=df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\4268819930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed']=df['player_dismissed'].astype('int')
C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\4268819930.py:6: FutureWarning: The default value of numeric_only in DataFrameGroup

In [22]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1
245,1795,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1
246,1795,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1
247,1795,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1


In [23]:
# now we focus on current run rate
df['crr']=(df['current_score']*6)/df['balls_bowled']

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\2918302938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crr']=(df['current_score']*6)/df['balls_bowled']


In [24]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10,0.000000
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10,0.000000
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10,0.000000
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10,0.000000
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10,1.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1,4.870293
245,1795,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1,4.850000
246,1795,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1,4.854772
247,1795,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1,4.854772


In [25]:
# now we focus on last ten over runs
groups=df.groupby('match_id')
match_ids=df['match_id'].unique()
last_ten=[]
for id in match_ids:
    last_ten.extend(groups.get_group(id).rolling(window=60).sum()['runs'].values.tolist())

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\2371810688.py:6: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city'], dtype='object')
  last_ten.extend(groups.get_group(id).rolling(window=60).sum()['runs'].values.tolist())


In [26]:
last_ten

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 56.0,
 60.0,
 60.0,
 61.0,
 62.0,
 61.0,
 61.0,
 61.0,
 61.0,
 60.0,
 61.0,
 62.0,
 62.0,
 62.0,
 61.0,
 62.0,
 61.0,
 60.0,
 61.0,
 61.0,
 62.0,
 62.0,
 62.0,
 58.0,
 59.0,
 59.0,
 58.0,
 57.0,
 57.0,
 57.0,
 57.0,
 58.0,
 58.0,
 57.0,
 56.0,
 56.0,
 55.0,
 56.0,
 56.0,
 56.0,
 57.0,
 56.0,
 54.0,
 55.0,
 56.0,
 50.0,
 50.0,
 49.0,
 50.0,
 46.0,
 47.0,
 46.0,
 46.0,
 43.0,
 39.0,
 35.0,
 36.0,
 36.0,
 36.0,
 36.0,
 33.0,
 30.0,
 32.0,
 32.0,
 32.0,
 33.0,
 33.0,
 33.0,
 34.0,
 34.0,
 35.0,
 34.0,
 34.0,
 38.0,
 38.0,
 38.0,
 40.0,
 40.0,
 39.0,
 40.0,
 39.0,
 40.0,
 39.0,
 43.0,
 41.0,
 42.0,
 42.0,
 43.0,
 43.0,
 43.0,
 44.0,
 47.0,
 4

now for the first 10 overs of every match this list will give null values as it is impossible to get previous 10 over runs for the first 10 overs of the match

In [27]:
df['last_ten']=last_ten

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\2901210300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_ten']=last_ten


In [28]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_ten
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,299,10,0.000000,NaN
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,298,10,0.000000,NaN
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,297,10,0.000000,NaN
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,296,10,0.000000,NaN
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,295,10,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1795,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,61,1,4.870293,53.0
245,1795,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,60,1,4.850000,49.0
246,1795,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,59,1,4.854772,50.0
247,1795,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,59,1,4.854772,50.0


In [29]:
# we want final score also
final_df=df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')
# runs_x represents the final score
# runs_y represents the score on particular ball

C:\Users\Soumadeep\AppData\Local\Temp\ipykernel_19044\474038776.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df=df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')


In [30]:
final_df=final_df[['batting_team','bowling_team','city',
   'current_score','balls_left',
   'wickets_left','crr','last_ten',
         'runs_x']]

In [31]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
0,Australia,Pakistan,Brisbane,0,299,10,0.000000,NaN,268
1,Australia,Pakistan,Brisbane,0,298,10,0.000000,NaN,268
2,Australia,Pakistan,Brisbane,0,297,10,0.000000,NaN,268
3,Australia,Pakistan,Brisbane,0,296,10,0.000000,NaN,268
4,Australia,Pakistan,Brisbane,1,295,10,1.200000,NaN,268
...,...,...,...,...,...,...,...,...,...
276739,Sri Lanka,Australia,Pallekele,194,61,1,4.870293,53.0,195
276740,Sri Lanka,Australia,Pallekele,194,60,1,4.850000,49.0,195
276741,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195
276742,Sri Lanka,Australia,Pallekele,195,59,1,4.854772,50.0,195


In [32]:
#there is some null values
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_ten         55995
runs_x               0
dtype: int64

In [33]:
final_df.dropna(inplace=True)

In [34]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_ten         0
runs_x           0
dtype: int64

There is no missing values Now

In [35]:
# to avoid the creating of biasedness we shuffle our dataframe
final_df=final_df.sample(final_df.shape[0])

In [36]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten,runs_x
247696,South Africa,New Zealand,Mount Maunganui,220,59,8,5.477178,64.0,282
60278,England,Australia,Manchester,127,99,5,3.791045,34.0,231
106218,Australia,India,Hyderabad,153,110,7,4.831579,36.0,290
246509,Sri Lanka,Pakistan,Hambantota,91,181,6,4.588235,46.0,275
195194,Pakistan,Sri Lanka,Colombo,201,51,6,4.843373,59.0,247
...,...,...,...,...,...,...,...,...,...
80116,South Africa,Australia,Durban,162,83,6,4.479263,54.0,246
156916,India,Sri Lanka,Mirpur,137,149,7,5.443709,48.0,279
182127,Sri Lanka,Australia,Melbourne,131,139,8,4.881988,60.0,238
18735,New Zealand,England,Christchurch,93,141,4,3.509434,33.0,223


# now we go to the model building part


In [37]:
x=final_df.drop(columns=['runs_x'])
y=final_df['runs_x']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=18)

In [38]:
x_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_ten
79177,Australia,South Africa,Centurion,124,117,7,4.065574,56.0
11972,England,South Africa,Southampton,250,43,6,5.836576,75.0
93407,Australia,New Zealand,Auckland,103,165,8,4.577778,51.0
160225,England,Australia,Melbourne,250,46,4,5.905512,64.0
214407,Pakistan,South Africa,Dubai,112,137,8,4.122699,45.0
...,...,...,...,...,...,...,...,...
267234,Sri Lanka,England,Birmingham,159,105,7,4.892308,56.0
4128,New Zealand,South Africa,Hamilton,111,169,5,5.083969,48.0
252028,Sri Lanka,England,Pallekele,27,240,9,2.700000,26.0
138620,Australia,India,Centurion,221,60,7,5.525000,62.0


In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error



In [40]:
trf=ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],remainder='passthrough')

In [41]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.1,max_depth=12,random_state=18))
])

In [42]:
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
a=r2_score(y_test,y_pred)
b=mean_absolute_error(y_test,y_pred)

C:\Users\Soumadeep\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [43]:
a

0.9926423493776517

In [44]:
b

2.236405554963098

In [45]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [46]:
eligible_cities

['Mirpur',
 'London',
 'Colombo',
 'Sydney',
 'Abu Dhabi',
 'Rangiri',
 'Melbourne',
 'Centurion',
 'Adelaide',
 'Dubai',
 'Birmingham',
 'Perth',
 'Auckland',
 'Cardiff',
 'Pallekele',
 'Brisbane',
 'Wellington',
 'Johannesburg',
 'Hamilton',
 'Sharjah',
 'Chandigarh',
 'Cape Town',
 'Durban',
 'Southampton',
 'Nottingham',
 'Manchester',
 'Port Elizabeth',
 'Leeds',
 'Karachi',
 'Napier',
 'Christchurch',
 'Hambantota',
 'Hobart',
 'Chester-le-Street',
 'Nagpur',
 'Mumbai',
 'Lahore',
 'Mount Maunganui',
 'Chittagong',
 'Delhi',
 'Fatullah',
 'Dunedin',
 'Rajkot',
 'Jaipur',
 'Kolkata',
 'Nelson',
 'Ahmedabad',
 'Hyderabad',
 'Bristol',
 'Canberra',
 'Harare',
 'Kanpur',
 'Bloemfontein']